# MLIS Judge codebase

In [0]:
# File: /usr/src/mlis-pytorch/mlis/core/case_data.py
class CaseData:
    REGRESSION = 'regression'
    CLASSIFICATION = 'classification'

    def __init__(self):
        self.type = self.CLASSIFICATION

    def set_type(self, type):
        self.type = type

    def set_number(self, number):
        self.number = number

    def set_train_data(self, train_data):
        self.train_data = train_data

    def set_test_data(self, test_data):
        self.test_data = test_data

    def set_run_seed(self, run_seed):
        self.run_seed = run_seed

    def set_limits(self, limits):
        self.limits = limits

    def set_description(self, description):
        self.description = description
        return self

    def get_limits(self):
        return self.limits


# File: /usr/src/mlis-pytorch/mlis/core/config.py
class Config:
    pass

# File: /usr/src/mlis-pytorch/mlis/core/limits.py
class Limits:
    def __init__(self, limits_config):
        self.training_time_limit = limits_config['trainingTimeLimit']
        self.model_size_limit = limits_config.get('modelSizeLimit', None)
        self.train_accuracy_limit = limits_config.get('trainAccuracyLimit', None)
        self.test_accuracy_limit = limits_config.get('testAccuracyLimit', None)
        self.train_metric_limit = limits_config.get('trainMetricLimit', None)
        self.test_metric_limit = limits_config.get('testMetricLimit', None)

# File: /usr/src/mlis-pytorch/mlis/core/solution_tester.py
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#from .speed_calculator import SpeedCalculator
#from .training_context import TrainingContext
#from .timer import Timer
#from .limits import Limits

class SolutionTester():
    HINT_YELLOW = '\033[93m'
    ACCEPTED_GREEN = '\033[92m'
    REJECTED_RED = '\033[91m'
    END_COLOR = '\033[0m'
    def __init__(self):
        self.mse_loss = nn.MSELoss()

    def calc_model_size(self, model):
        modelSize = 0
        for param in model.parameters():
            modelSize += param.view(-1).size()[0]
        return modelSize

    def sample_data(self, data, max_samples = 10000):
        return data[:max_samples]

    def metric(self, output, target):
        if isinstance(target, list):
            target = torch.FloatTensor(target).view_as(output)
        return self.mse_loss(output, target).item()

    def calc_correct(self, predict, target):
        if isinstance(target, list):
            return sum([p == t for p, t in zip(predict, target)])
        else:
            return predict.eq(target.view_as(predict)).long().sum().item()

    def calc_model_stats(self, case_data, model, data, target, time_mult):
        with torch.no_grad():
            data = self.sample_data(data)
            target = self.sample_data(target)
            evaluation_start_time = time.time()
            output = model(data)
            evaluation_end_time = time.time()
            # Number of correct predictions
            error = model.calc_error(output, target).item()
            total = len(data)
            metric = self.metric(output, target)
            if case_data.type == case_data.CLASSIFICATION:
                predict = model.calc_predict(output)
                correct = self.calc_correct(predict, target)
                accuracy = correct/total
            else:
                correct = None
                accuracy = None

            return {
                    'error': error,
                    'correct': correct,
                    'total': total,
                    'accuracy': accuracy,
                    'metric': metric,
                    'evaluation_time': (evaluation_end_time - evaluation_start_time)*time_mult,
                    }

    def train_model(self, solution, train_data, context):
        # We need to init random system, used for multiple runs
        torch.manual_seed(context.run_seed)
        model = solution.train_model(*train_data, context)
        return context.step, model

    def set_case_data_from_config(self, case_data, test_config):
        case_data.set_number(test_config['number'])
        case_data.set_run_seed(test_config['runSeed'])
        case_data.set_description(test_config['description'])
        case_data.set_limits(Limits(test_config['limits']))
        return case_data

    def get_case_data_from_test_config(self, config, test_config):
        data_provider = config.get_data_provider()
        data_provider_config = test_config['dataProviderConfig']
        case_data = data_provider.create_case_data(data_provider_config)
        case_data = self.set_case_data_from_config(case_data, test_config)
        return case_data

    def run_case_from_case_data(self, config, case_data, time_mult = 1.0):
        limits = case_data.get_limits()
        timer = Timer(limits.training_time_limit, time_mult)
        context = TrainingContext(case_data.run_seed, timer)
        training_start_time = time.time()
        solution = config.get_solution()
        step, model = self.train_model(solution, case_data.train_data, context)
        training_end_time = time.time()
        training_time = (training_end_time - training_start_time)*time_mult
        model_size = self.calc_model_size(model)
        model.eval()
        train_stat = self.calc_model_stats(case_data, model, *case_data.train_data, time_mult)
        test_stat = self.calc_model_stats(case_data, model, *case_data.test_data, time_mult)

        return {
            'modelSize': model_size,
            'trainingSteps': step,
            'trainingTime': training_time,
            'trainEvaluationTime': train_stat['evaluation_time'],
            'trainError': train_stat['error'],
            'trainMetric': train_stat['metric'],
            'trainCorrect': train_stat['correct'],
            'trainTotal': train_stat['total'],
            'trainAccuracy': train_stat['accuracy'],
            'testEvaluationTime': test_stat['evaluation_time'],
            'testError': test_stat['error'],
            'testMetric': test_stat['metric'],
            'testCorrect': test_stat['correct'],
            'testTotal': test_stat['total'],
            'testAccuracy': test_stat['accuracy'],
        }

    def run_case_from_test_config(self, config, test_config, time_mult = 1.0):
        case_data = self.get_case_data_from_test_config(config, test_config)
        return self.run_case_from_case_data(config, case_data, time_mult)

    @classmethod
    def colored_string(self, s, color):
        return color+s+SolutionTester.END_COLOR

    @classmethod
    def print_hint(self, s, step=0):
        if step==0:
            print(SolutionTester.colored_string(s, SolutionTester.HINT_YELLOW))


    def hint_string(self, s):
        return self.colored_string(s, SolutionTester.HINT_YELLOW)

    def rejected_string(self, s):
        return self.colored_string(s, SolutionTester.REJECTED_RED)

    def accepted_string(self, s):
        return self.colored_string(s, SolutionTester.ACCEPTED_GREEN)

    def evaluate_result(self, case_data, case_result):
        limits = case_data.get_limits()
        r = case_result
        case = case_data.number
        description = case_data.description
        size = r['modelSize']
        step = r['trainingSteps']
        time = r['trainingTime']
        train_error = r['trainError']
        train_metric = r['trainMetric']
        train_correct = r['trainCorrect']
        train_total = r['trainTotal']
        train_accuracy = r['trainAccuracy']
        test_error = r['testError']
        test_metric = r['testMetric']
        test_correct = r['testCorrect']
        test_total = r['testTotal']
        test_accuracy = r['testAccuracy']

        print("Case #{}[{}] Step={} Size={}/{} Time={:.1f}/{:.1f}".format(
            case, description, step, size, limits.model_size_limit, time, limits.training_time_limit))
        if train_metric is not None:
            print("Train metric/limit={:.4f}/{} Error={}".format(
                train_metric, limits.train_metric_limit, train_error))
        if test_metric is not None:
            print("Test  metric/limit={:.4f}/{} Error={}".format(
                test_metric, limits.test_metric_limit, test_error))
        if train_accuracy is not None:
            print("Train correct/total={}/{} Ratio/limit={:.2f}/{:.2f} Error={}".format(
                train_correct, train_total, train_accuracy, limits.train_accuracy_limit, train_error))
        if test_accuracy is not None:
            print("Test  correct/total={}/{} Ratio/limit={:.2f}/{:.2f} Error={}".format(
                test_correct, test_total, test_accuracy, limits.test_accuracy_limit, test_error))
        r['accepted'] = False
        if limits.model_size_limit is not None and size > limits.model_size_limit:
            print(self.rejected_string("[REJECTED]")+": MODEL IS TOO BIG: Size={} Size Limit={}".format(size, limits.model_size_limit))
        elif time > limits.training_time_limit:
            print(self.rejected_string("[REJECTED]")+": TIME LIMIT EXCEEDED: Time={:.1f} Time Limit={:.1f}".format(time, limits.training_time_limit))
        elif limits.test_accuracy_limit is not None and test_accuracy < limits.test_accuracy_limit:
            print(self.rejected_string("[REJECTED]")+": MODEL DID NOT LEARN: Learn ratio={}/{}".format(test_accuracy, limits.test_accuracy_limit))
        elif limits.train_metric_limit is not None and train_metric > limits.train_metric_limit:
            print(self.rejected_string("[REJECTED]")+": MODEL DID NOT LEARN: Train metric={}/{}".format(train_metric, limits.train_metric_limit))
        elif limits.test_metric_limit is not None and test_metric > limits.test_metric_limit:
            print(self.rejected_string("[REJECTED]")+": MODEL DID NOT GENERELIZE: Test metric={}/{}".format(test_metric, limits.test_metric_limit))
        else:
            r['accepted'] = True
            print(self.accepted_string("[OK]"))

        return r

    def get_tests_with_limits(self, test_set, case_number):
        limits = test_set.get('limits', None)
        tests = test_set['tests']
        for test in tests:
            if ('limits' not in test):
                test['limits'] = limits
        if case_number != -1:
            tests = [test for test in tests if test['number'] == case_number]
        return tests

    def run(self, config, test_set, case_number):
        tests = self.get_tests_with_limits(test_set, case_number)
        speed_calculator = SpeedCalculator()
        time_mult = speed_calculator.calc_linear_time_mult()
        print("Local CPU time mult = {:.2f}".format(time_mult))
        case_results = []
        case_limits = []
        for test_config in tests:
            case_data = self.get_case_data_from_test_config(config, test_config)
            case_result = self.run_case_from_case_data(config, case_data, time_mult)
            case_result = self.evaluate_result(case_data, case_result)
            if case_result['accepted'] == False:
                print("Need more hint??? Ask for hint at Facebook comments")
                return False
            case_limits.append(case_data.get_limits())
            case_results.append(case_result)

        num_cases = float(len(case_results))
        if case_results[0]['testAccuracy'] is not None:
            test_rates = [x['testAccuracy'] for x in case_results]
            test_rate_max = max(test_rates)
            test_rate_mean = sum(test_rates)/len(test_rates)
            test_rate_min = min(test_rates)
            test_limit_mean = sum([x.test_accuracy_limit for x in case_limits])/num_cases
            print("Test rate (max/mean/min/limit) = {:.3f}/{:.3f}/{:.3f}/{:.3f}".format(
                test_rate_max, test_rate_mean, test_rate_min, test_limit_mean))
        step_mean = sum([x['trainingSteps'] for x in case_results])/num_cases
        time_mean = sum([x['trainingTime'] for x in case_results])/num_cases
        size_mean = sum([x['modelSize'] for x in case_results])/num_cases
        time_limit_mean = sum([x.training_time_limit for x in case_limits])/num_cases
        #size_limit_mean = sum([x.model_size_limit for x in case_limits])/num_cases
        print("Average steps = {:.3f}".format(step_mean))
        print("Average time = {:.3f}/{:.3f}".format(time_mean, time_limit_mean))
        print("Average size = {:.3f}".format(size_mean))
        if case_number == -1:
            print(self.accepted_string("[ACCEPTED]")+" you can submit now your score")
            print("Go to https://www.mlisjudge.com to submit your code")
        else:
            print(self.hint_string("[GOOD]")+" test passed, try to run on all tests")



# File: /usr/src/mlis-pytorch/mlis/core/speed_calculator.py
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class LinearModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(LinearModel, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.sigmoid(x)
        x = self.linear2(x)
        x = torch.sigmoid(x)
        x = self.linear3(x)
        x = torch.sigmoid(x)
        return x

class ConvolModel(nn.Module):
    def __init__(self):
        super(ConvolModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 5, 2)
        self.conv2 = nn.Conv2d(5, 10, 2)
        self.conv3 = nn.Conv2d(10, 10, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.max_pool2d(self.conv3(x), 2)
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(x)
        return x

class SpeedCalculator:
    def calc_linear_time_mult(self, use_gpu = False):
        batch_size = 256
        number_of_batches = 100
        expected_steps_per_second = 1000.0
        if use_gpu:
            batch_size *= 10
            expected_steps_per_second /= 10
            number_of_batches *= 10
        input_size = 10
        output_size = 10
        hidden_size = 100
        data = torch.FloatTensor(batch_size*number_of_batches, input_size)
        target = torch.FloatTensor(batch_size*number_of_batches, output_size)
        model = LinearModel(input_size, output_size, hidden_size)
        if use_gpu:
            model = model.cuda()
            data = data.cuda()
            target = target.cuda()
        torch.manual_seed(1)
        data.uniform_(-1.0, 1.0)
        target.uniform_(-1.0, 1.0)

        optimizer = optim.SGD(model.parameters(), lr=0.00001)
        start_time = time.time()
        for ind in range(number_of_batches):
            data_batch = data[batch_size*ind:batch_size*(ind+1)]
            target_batch = target[batch_size*ind:batch_size*(ind+1)]
            optimizer.zero_grad()
            output = model(data_batch)
            loss = F.mse_loss(output, target_batch)
            loss.backward()
            # We don't do step for stability
            # optimizer.step()
        end_time = time.time()
        steps_per_second = number_of_batches/(end_time-start_time)
        return steps_per_second/expected_steps_per_second

    def calc_convol_time_mult(self, use_gpu = False):
        batch_size = 256
        number_of_batches = 100
        expected_steps_per_second = 48.0
        if use_gpu:
            batch_size *= 10
            expected_steps_per_second /= 10
            number_of_batches *= 10
        input_size = 10
        output_size = 40
        hidden_size = 100
        data = torch.FloatTensor(batch_size*number_of_batches, 1, 28, 28)
        target = torch.FloatTensor(batch_size*number_of_batches, output_size)
        model = ConvolModel()
        if use_gpu:
            model = model.cuda()
            data = data.cuda()
            target = target.cuda()
        torch.manual_seed(1)
        data.uniform_(-1.0, 1.0)
        target.uniform_(-1.0, 1.0)

        optimizer = optim.SGD(model.parameters(), lr=0.00001)
        start_time = time.time()
        for ind in range(number_of_batches):
            data_batch = data[batch_size*ind:batch_size*(ind+1)]
            target_batch = target[batch_size*ind:batch_size*(ind+1)]
            optimizer.zero_grad()
            output = model(data_batch)
            loss = F.mse_loss(output, target_batch)
            loss.backward()
            # We don't do step for stability
            # optimizer.step()
        end_time = time.time()
        steps_per_second = number_of_batches/(end_time-start_time)
        return steps_per_second/expected_steps_per_second

class SpeedTest():
    def __init__(self):
        self = self

    def print_speed_report(self):
        speed_calculator = SpeedCalculator()
        linear_time_mult = speed_calculator.calc_linear_time_mult()
        print("Linear CPU time mult = {:.2f}".format(linear_time_mult))
        convol_time_mult = speed_calculator.calc_convol_time_mult()
        print("Convol CPU time mult = {:.2f}".format(convol_time_mult))
        if torch.cuda.is_available():
            linear_time_mult = speed_calculator.calc_linear_time_mult(True)
            print("Linear GPU time mult = {:.2f}".format(linear_time_mult))
            convol_time_mult = speed_calculator.calc_convol_time_mult(True)
            print("Convol GPU time mult = {:.2f}".format(convol_time_mult))
        else:
            print("No cuda")


# File: /usr/src/mlis-pytorch/mlis/core/tester_config.py
class TesterConfig:
    def __init__(self, data_provider, solution):
        self.data_provider = data_provider
        self.solution = solution

    def get_data_provider(self):
        return self.data_provider()

    def get_solution(self):
        return self.solution()

# File: /usr/src/mlis-pytorch/mlis/core/timer.py
import time
from contextlib import contextmanager

class Timer():
    def __init__(self, time_limit, time_mult):
        self.time_limit = time_limit
        self.time_mult = time_mult
        self.start_time = time.time()
        self.pause_time = 0.0

    @contextmanager
    def pause(self):
        pause_start = time.time()
        yield None
        pause_end = time.time()
        self.pause_time += pause_end-pause_start

    def get_time_left(self):
        return self.time_limit - self.get_execution_time()

    def get_execution_time(self):
        return (time.time() - self.start_time - self.pause_time) * self.time_mult

    def get_pause_time(self):
        return self.pause_time


# File: /usr/src/mlis-pytorch/mlis/core/training_context.py
class TrainingContext():
    LOCAL_TRAINING = 'local_training'
    SERVER_TRAINING = 'server_training'
    GRID_SEARCH = 'grid_search'
    def __init__(self, run_seed, timer):
        self.type = self.__class__.LOCAL_TRAINING
        self.run_seed = run_seed
        self.timer = timer
        self.step = 0

    def increase_step(self):
        self.step += 1# File: /usr/src/mlis-pytorch/mlis/utils/__init__.py


# File: /usr/src/mlis-pytorch/mlis/utils/grid_search.py
import random
import pickle
import os
import sys
import torch
import inspect
import collections
import pandas as pd
#from ..core.timer import Timer
#from ..core.training_context import TrainingContext
#from ..core.solution_tester import SolutionTester
#from ..core.speed_calculator import SpeedCalculator

def new_getfile(object, _old_getfile=inspect.getfile):
    try:
        return _old_getfile(object)
    except:
        # If parent module is __main__, lookup by methods (NEW)
        for name, member in inspect.getmembers(object):
            if inspect.isfunction(member) and object.__qualname__ + '.' + member.__name__ == member.__qualname__:
                return inspect.getfile(member)
        raise TypeError('Source for {!r} not found'.format(object))
inspect.getfile = new_getfile

class RunsLogs():
    def __init__(self):
        self.run_params_keys = []
        self.run_params_values_sortable = []
        self.clear_data()

    def clear_data(self):
        self.run_params_to_scalars = {}

    def is_sortable(self, l):
        try:
            sorted(l)
            return True
        except:
            return False

    def set_run_params_grid(self, run_params_grid):
        run_params_keys = list(run_params_grid.keys())
        if sorted(self.run_params_keys) != sorted(run_params_keys):
            print("[WARNING] run_params keys changed, clearning data")
            self.clear_data()
        self.run_params_keys = run_params_keys
        self.run_params_values_sortable = [self.is_sortable(values) for values in run_params_grid.values()]

    def get_next_run_seed(self, run_params):
        next_run_seed = 0
        for name, values in self.run_params_to_scalars.get(run_params, {}).items():
            next_run_seed = max(next_run_seed, max(values.keys(), default=-1)+1)
        return next_run_seed

    def log_scalar(self, run_params, name, run_seed, value):
        self.run_params_to_scalars.setdefault(run_params, {}).setdefault(name, {})[run_seed] = value

    def get_scalars(self, run_params, name):
        return list(self.run_params_to_scalars[run_params][name].values())

    def save(self, file_name):
        file_pi = open(file_name, 'wb')
        pickle.dump(self, file_pi)

    def get_value_types(self):
        columns = []
        columns_dict = {}
        for key, value in self.run_params_to_scalars.items():
            for column, _ in value.items():
                if column not in columns_dict:
                    columns.append(column)
                    columns_dict[column] = True
        return columns

    def get_columns(self):
        return self.run_params_keys.copy()

    def get_dataframe(self):
        data = []
        for run_params, name_to_scalars in self.run_params_to_scalars.items():
            datum = [getattr(run_params, run_param_key) for run_param_key in self.run_params_keys]
            datum = [d if sortable else str(d) for sortable, d in zip(self.run_params_values_sortable, datum)]
            for name, scalars in name_to_scalars.items():
                for value in scalars.values():
                    datum_all = datum.copy()
                    datum_all.append(name)
                    datum_all.append(value)
                    data.append(datum_all)

        columns = self.run_params_keys + ['name', 'value']
        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def load(file_name):
        if os.path.isfile(file_name):
            try:
                return pickle.load(open(file_name, 'rb'))
            except Exception as e:
                print(e)
                print("[WARNING] file with data can not be loaded, return empty results data")
        else:
            print("[WARNING] file with data not exists, return empty results data")
        return RunsLogs()

    @staticmethod
    def get_global():
        global RESULTS_DATA_INSTANCE
        if 'RESULTS_DATA_INSTANCE' not in globals():
            RESULTS_DATA_INSTANCE = RunsLogs()
        return RESULTS_DATA_INSTANCE

class GridSearchConfig():
    RUN_COUNT = 'runs_per_params'
    def __init__(self):
        self.test_config = None
        self.random_order = False
        self.verbose = False

    def set_test_config(self, test_config):
        self.test_config = test_config
        return self

    def set_random_order(self, random_order):
        self.random_order = random_order
        return self

    def set_verbose(self, verbose):
        self.verbose = verbose
        return self

    def set_runs_config_from_command_line(self, parser):
        all_args = ';'.join(sys.argv)
        runs_per_params = 1
        args = parser.parse_args()
        runs_params_grid = {}
        sorted_attributes = []
        for key, value in vars(args).items():
            if key == self.__class__.RUN_COUNT:
                runs_per_params = value
            else:
                index = all_args.find(key)
                if index == -1:
                    runs_params_grid[key] = value
                else:
                    sorted_attributes.append((index, key, value))
        for _, key, value in sorted(sorted_attributes):
            runs_params_grid[key] = value

        self.set_runs_config(runs_params_grid, runs_per_params)

    def set_runs_config_from_solution(self, solution):
        runs_per_params = self.get_runs_per_params_from_solution(solution)
        s = solution
        code = inspect.getsource(type(s))
        grid_attrs =[(code.index(a), a) for a in dir(s) if self.filter_grid_attribute_key(s, a)]
        grid_attrs = sorted(grid_attrs)

        runs_params_grid = collections.OrderedDict()
        for _, a in grid_attrs:
            runs_params_grid[self.calc_grid_attribute_key(s, a)] = self.get_grid_attribute_list_from_solution(s, a)

        self.set_runs_config(runs_params_grid, runs_per_params)

    def set_runs_config(self, runs_params_grid, runs_per_params):
        self.runs_params_grid = runs_params_grid
        self.runs_per_params = runs_per_params

    def get_runs_per_params_from_solution(self, solution):
        if hasattr(solution, self.__class__.RUN_COUNT):
            return solution.runs_per_params
        else:
            return 1

    def calc_grid_attribute_key(self, obj, attr):
        return attr[:-len(GridSearch.GRID_LIST_SUFFIX)]

    def get_grid_attribute_list_from_solution(self, solution, attribute_key):
        return getattr(solution, attribute_key)

    def filter_grid_attribute_key(self, obj, attr):
        return attr.endswith(GridSearch.GRID_LIST_SUFFIX) and not attr.startswith('__') and not callable(getattr(obj,attr))

class GridSearchContext(TrainingContext):
    def __init__(self, run_seed, timer, case_data, run_idx_per_params, runs_per_params, run_params, runs_logs):
        super(GridSearchContext, self).__init__(run_seed, timer)
        self.type = self.__class__.GRID_SEARCH
        self.case_data = case_data
        self.run_params = run_params
        self.runs_per_params = runs_per_params
        self.run_idx_per_params = run_idx_per_params
        self.runs_logs = runs_logs

    def log_scalar(self, name, value):
        self.runs_logs.log_scalar(self.run_params, name, self.run_seed, value)

    def get_scalars(self, name):
        return self.runs_logs.get_scalars(self.run_params, name)

    def get_scalars_stats(self, name):
        results = self.get_scalars(name)
        t = torch.FloatTensor(results)
        return dict(mean=t.mean().item(), std=t.std().item())

    def __str__(self):
        main_str = self.__class__.__name__ + '(\n'
        for key, value in vars(self).items():
            main_str += "  {}: {}\n".format(key, value)
        main_str += ')'
        return main_str

class RunParams():
    SHORT_PARAM_SEPARATOR = ' '
    SHORT_VALUE_SEPARATOR = ':'

    def __init__(self, params):
        self.__dict__.update(params)
        self.__key_cache = self.to_string(sorted_keys=True)

    def __hash__(self):
        return hash(self.__key_cache)

    def __eq__(self, other):
        return other.__key_cache == self.__key_cache

    def __str__(self):
        return self.to_string()

    def to_string(self, sorted_keys=False):
        param_separator = self.__class__.SHORT_PARAM_SEPARATOR
        value_separator = self.__class__.SHORT_VALUE_SEPARATOR
        items = self.__dict__.items()
        items = [(key, value) for key, value in items if not key.startswith('_RunParams__')]
        items = sorted(items) if sorted_keys else reversed(list(items))
        return param_separator.join(
                [key+value_separator+repr(value) for key, value in items])

class GridSearch():
    GRID_LIST_SUFFIX = '_grid'
    GRID_PARAM_SEPARATOR = ' '
    GRID_VALUE_SEPARATOR = ':'


    def calc_grid_size(self, runs_params_grid):
        grid_size = 1
        for attr, attr_list in runs_params_grid.items():
            grid_size *= len(attr_list)
        return grid_size

    def get_run_params(self, runs_params_grid, grid_runs_history, random_order):
        history_size = len(grid_runs_history)
        while True:
            grid_choice = {}
            choice_ind = history_size
            for attr, attr_list in reversed(list(runs_params_grid.items())):
                attr_list_size = len(attr_list)
                if random_order:
                    attr_ind = random.randint(0, attr_list_size-1)
                else:
                    attr_ind = choice_ind%attr_list_size
                    choice_ind //= attr_list_size
                grid_choice[attr] = attr_list[attr_ind]
            run_params = RunParams(grid_choice)
            if run_params not in grid_runs_history:
                return run_params

    def check_runs_params_grid(self, runs_params_grid):
        for key, value in runs_params_grid.items():
            unique_len = len(set([str(x) for x in value]))
            if len(value) != unique_len:
                raise ValueError('Non unique attributes: {} = {}'.format(key, value))

    def run(self, tester_config, config, runs_logs):
        speed_calculator = SpeedCalculator()
        time_mult = speed_calculator.calc_linear_time_mult()
        print("Local CPU time mult = {:.2f}".format(time_mult))

        self.check_runs_params_grid(config.runs_params_grid)
        runs_logs.set_run_params_grid(config.runs_params_grid)
        grid_size = self.calc_grid_size(config.runs_params_grid)
        if config.verbose:
            print('[Grid search] Runing: grid_size={} runs_per_params={} verbose={}'.format(grid_size, config.runs_per_params, config.verbose))
        grid_runs_history = {}
        solution_tester = SolutionTester()
        solution = tester_config.get_solution()
        case_data = solution_tester.get_case_data_from_test_config(tester_config, config.test_config)
        while len(grid_runs_history) <  grid_size:
            run_params = self.get_run_params(config.runs_params_grid, grid_runs_history, config.random_order)
            for run_idx_per_params in range(config.runs_per_params):
                if config.verbose:
                    print('[Grid search] Running: run_params={} run_idx_per_params={}'.format(run_params, run_idx_per_params))
                self.run_seed = runs_logs.get_next_run_seed(run_params)
                limits = case_data.get_limits()
                timer = Timer(limits.training_time_limit, time_mult)
                context = GridSearchContext(
                        self.run_seed, timer, case_data, run_idx_per_params, config.runs_per_params, run_params, runs_logs)
                solution_tester.train_model(solution, case_data.train_data, context)

            grid_runs_history[run_params] = True
        print(solution_tester.accepted_string("[SEARCH COMPLETED]"))
        return runs_logs


# File: /usr/src/mlis-pytorch/mlis/utils/plotter.py
import matplotlib.pyplot as plt
import seaborn as sns

class Plotter():
    COLUMN_DELIMITER = ':'
    def __init__(self, results_data):
        self.results_data = results_data

    def show_1d(self, column_index = -1, query = None):
        columns = self.results_data.get_columns()

        df = self.results_data.get_dataframe()
        if query is not None:
            df.query(query, inplace=True)

        if df.shape[0] == 0:
            print('[WARNING] Empty filtered results')
            return

        if column_index == -1:
            uniq_per_column = df[columns].nunique()
            for i, r in enumerate(uniq_per_column):
                if r > 1:
                    column_index = i
                    break
        x_column = columns.pop(column_index)
        col_column = 'col:'+Plotter.COLUMN_DELIMITER.join(columns)
        df[col_column] = df.apply (lambda row: Plotter.COLUMN_DELIMITER.join([str(row[c]) for c in columns]), axis=1)
        sns.set(style="darkgrid")
        sns.relplot(x=x_column, y="value", hue="name", col=col_column, col_wrap=2, kind="line", markers=True, data=df)
        plt.show()


# Problem

In [0]:
TestSet = {
    "name": "TestSet1",
    "tests": [
        {
            "number": 1,
            "runSeed": 1,
            "description": "hola vs hello",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "languages": [
                    {
                        "minWordsCount": 1,
                        "maxWordsCount": 1,
                        "words": [
                            "hola"
                        ]
                    },
                    {
                        "minWordsCount": 1,
                        "maxWordsCount": 1,
                        "words": [
                            "hello"
                        ]
                    }
                ]
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 2,
            "runSeed": 2,
            "description": "ab vs ba 1-3",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "languages": [
                    {
                        "minWordsCount": 1,
                        "maxWordsCount": 3,
                        "words": [
                            "ab",
                            "bac"
                        ]
                    },
                    {
                        "minWordsCount": 1,
                        "maxWordsCount": 3,
                        "words": [
                            "ba",
                            "abc"
                        ]
                    }
                ]
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 3,
            "runSeed": 3,
            "description": "ab vs ba 1-10",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "languages": [
                    {
                        "minWordsCount": 1,
                        "maxWordsCount": 10,
                        "words": [
                            "ab",
                            "bac"
                        ]
                    },
                    {
                        "minWordsCount": 1,
                        "maxWordsCount": 10,
                        "words": [
                            "ba",
                            "abc"
                        ]
                    }
                ]
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 4,
            "runSeed": 4,
            "description": "ab vs ba 10-20",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "languages": [
                    {
                        "minWordsCount": 10,
                        "maxWordsCount": 20,
                        "words": [
                            "ab",
                            "bac"
                        ]
                    },
                    {
                        "minWordsCount": 10,
                        "maxWordsCount": 20,
                        "words": [
                            "ba",
                            "abc"
                        ]
                    }
                ]
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 5,
            "runSeed": 5,
            "description": "8 inputs per voter and 5 voters",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "inputSize": 8,
                "inputCountSize": 5
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 6,
            "runSeed": 6,
            "description": "8 inputs per voter and 6 voters",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "inputSize": 8,
                "inputCountSize": 6
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 7,
            "runSeed": 7,
            "description": "8 inputs per voter and 7 voters",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "inputSize": 8,
                "inputCountSize": 7
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 8,
            "runSeed": 8,
            "description": "8 inputs per voter and 8 voters",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "inputSize": 8,
                "inputCountSize": 8
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 9,
            "runSeed": 9,
            "description": "8 inputs per voter and 1 voters",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "inputSize": 8,
                "inputCountSize": 9
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        },
        {
            "number": 10,
            "runSeed": 10,
            "description": "8 inputs per voter and 10 voters",
            "dataProviderConfig": {
                "seed": 1,
                "dataSize": 8192,
                "inputSize": 8,
                "inputCountSize": 10
            },
            "limits": {
                "trainingTimeLimit": 2,
                "trainEvaluationTimeLimit": 2,
                "testEvaluationTimeLimit": 2,
                "trainAccuracyLimit": 1,
                "testAccuracyLimit": 1
            }
        }
    ]
}
import torch
#from ..core.case_data import CaseData

# There are 2 languages.
class Language:
    def __init__(self, config):
        self.min_words_count = config['minWordsCount']
        self.max_words_count = config['maxWordsCount']
        self.words = config['words']

    def gen_sentences(self, number_of_sentences):
        sentences_length = torch.LongTensor(number_of_sentences).random_(self.min_words_count, self.max_words_count+1)
        total_length = sentences_length.sum().item()
        random_words_index = torch.LongTensor(total_length).random_(0, len(self.words))
        random_words = [self.words[word_index] for word_index in random_words_index.tolist()]
        sentences = []
        start_index = 0
        for sentence_length in sentences_length:
            sentences.append(''.join(random_words[start_index:start_index+sentence_length]))
            start_index += sentence_length
        return sentences
    
    def find_max_length_count(self, sentence):
        length_to_count = [0.0] * (len(sentence)+1)
        max_word_length = max([len(word) for word in self.words])
        words = set(self.words)
        length_to_count[0] = 1.0
        updated = True
        while updated:
            updated = False
            for i in range(len(length_to_count)-1, -1, -1):
                if length_to_count[i] > 0.0:
                    possible_words = sentence[i:i+max_word_length]
                    for word_len in range(1, max_word_length+1):
                        possible_word = possible_words[:word_len]
                        if possible_word in words:
                            length_to_count[i+word_len] += length_to_count[i]

class DataProvider:
    def __reindex_list(self, list_data, list_index):
        return [list_data[index] for index in list_index]

    def __create_data(self, data_size, languages, seed):
        torch.manual_seed(seed)
        language_indexes = torch.LongTensor(data_size).random_(0, len(languages))
        data = []
        target = []
        used_sentences = set()
        for language_index, language in enumerate(languages):
            number_of_sentences = (language_indexes == language_index).sum().item()
            sentences = language.gen_sentences(number_of_sentences)
            new_sentences = set(sentences)
            expected_size = len(used_sentences)+len(new_sentences)
            used_sentences = used_sentences.union(set(sentences))
            if len(used_sentences) != expected_size:
                raise "Not uniq sentence"
            data += sentences
            target += [language_index] * number_of_sentences

        perm = torch.randperm(len(data)).tolist()
        data = self.__reindex_list(data, perm)
        target = self.__reindex_list(target, perm)
        return (data, target)

    def create_case_data(self, config):
        case_data = CaseData()
        # correct seed help generate data faster
        seed = config['seed']
        data_size = config['dataSize']
        language_configs = config['languages']
        languages = [Language(language_config) for language_config in language_configs]
        data, target = self.__create_data(2*data_size, languages, seed)

        case_data.set_train_data((data[:data_size], target[:data_size]))
        case_data.set_test_data((data[data_size:], target[data_size:]))
        return case_data

Config.RuntimeName = 'notebook'
Config.DataProvider = DataProvider
Config.TestSet = TestSet

# Solution

In [0]:
# HelloXor is a HelloWorld of Machine Learning.
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf


class Hola(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(Hola, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.sigmoid(x)
        x = self.linear2(x)
        x = torch.sigmoid(x)
        return x

    def calc_error(self, output, target):
        # This is loss function
        return ((output-target)**2).sum()

    def calc_predict(self, output):
        # Simple round output to predict value
        return output.round()

class Solution():
    # Return trained model
    def train_model(self, train_data, train_target, context):
        # print("Hint[1]: Increase hidden size")
        hidden_size = 10
        # print("Hint[2]: Learning rate is too small")
        learning_rate = 0.01
        # Set up trainng parameters from grid search context
        if context.type == context.__class__.GRID_SEARCH:
            hidden_size, learning_rate = context.run_params.hidden_size, context.run_params.learning_rate
        # Model represent our neural network
        print("Train data is ",type(train_data)," Lenth ", len(train_data))
        print(train_data)
        print(train_target)
        train_data = tf.convert_to_tensor(train_data)
        print("converted train_data:")
        print("Train data is ",type(train_data))
        print(train_data)
        model = Hola(train_data.size(1), train_target.size(1), hidden_size)
        # Optimizer used for training neural network
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
        while True:
            # Report step, so we know how many steps
            context.increase_step()
            # model.parameters()...gradient set to zero
            optimizer.zero_grad()
            # evaluate model => model.forward(data)
            output = model(train_data)
            # if x < 0.5 predict 0 else predict 1
            predict = model.calc_predict(output)
            # Number of correct predictions
            correct = predict.eq(train_target.view_as(predict)).long().sum().item()
            # Total number of needed predictions
            total = predict.view(-1).size(0)
            # No more time left or learned everything, stop training
            time_left = context.timer.get_time_left()
            if time_left < 0.1 or correct == total:
                break
            # calculate error
            error = model.calc_error(output, train_target)
            # calculate deriviative of model.forward() and put it in model.parameters()...gradient
            error.backward()
            # print progress of the learning
            self.print_stats(context.step, error, correct, total)
            # update model: model.parameters() -= lr * gradient
            optimizer.step()
        # Log data for grid search
        self.grid_search_tutorial(context)
        return model

    def print_stats(self, step, error, correct, total):
        if step % 1000 == 0:
            print("Step = {} Correct = {}/{} Error = {}".format(step, correct, total, error.item()))

    def grid_search_tutorial(self, context):
        # During grid search, train_model will be called runs_per_params times with every possible combination of params.
        # This can be used for automatic parameters tunning.
        if context.type == context.__class__.GRID_SEARCH:
            print("[HelloXor] hidden_size={} learning_rate={} run_idx_per_params=[{}/{}]".format(
                context.run_params.hidden_size, context.run_params.learning_rate,
                context.run_idx_per_params, context.runs_per_params))
            time_left_key = 'time_left'
            step_key = 'step'
            context.log_scalar(time_left_key, context.timer.get_time_left())
            context.log_scalar(step_key, context.step)
            if context.run_idx_per_params == context.runs_per_params-1:
                print("[HelloXor] run_params={}".format(context.run_params))
                print("[HelloXor] time_left_logs={}".format(context.get_scalars(time_left_key)))
                print("[HelloXor] step_logs={}".format(context.get_scalars(step_key)))
                print("[HelloXor] step_logs_stats={}".format(context.get_scalars_stats(step_key)))


In [41]:
# The code below this line will not run on server
if __name__ != 'mlis.submission.solution':
    TESTER_CASE_NUMBER = 1
    TESTER_ENABLED = True
    GRID_SEARCH_CASE_NUMBER = 1
    GRID_SEARCH_ENABLED = False
    GRID_SEARCH_LOG_FILE = 'hola_runs_logs.pickle'

    #from ..core.config import Config
    #from ..core.tester_config import TesterConfig
    #from ..core.solution_tester import SolutionTester
    #from ..utils.grid_search import RunsLogs, GridSearchConfig, GridSearch
    #from ..utils.plotter import Plotter

    if TESTER_ENABLED:
        SolutionTester().run(TesterConfig(Config.DataProvider, Solution), Config.TestSet, TESTER_CASE_NUMBER)

    if GRID_SEARCH_ENABLED:
        tests = SolutionTester().get_tests_with_limits(Config.TestSet, GRID_SEARCH_CASE_NUMBER)
        grid_search_config = GridSearchConfig()
        grid_search_config.set_test_config(tests[0])
        grid_search_config.set_verbose(False)
        grid_search_config.set_runs_config(
            runs_params_grid = dict(
                hidden_size=[3, 4],
                learning_rate=[1.0,2.0,3.0],
            ),
            runs_per_params=10
        )
        # Uncomment to config grid search from solution
        # grid_search_config.set_runs_config_from_solution(Solution())

        # Note: we can save and accumulate results data if grid keys did not change
        runs_logs_file = GRID_SEARCH_LOG_FILE
        runs_logs = RunsLogs.load(runs_logs_file)
        runs_logs = GridSearch().run(TesterConfig(Config.DataProvider, Solution), grid_search_config, runs_logs)
        runs_logs.save(runs_logs_file)

        # Explore data
        df = runs_logs.get_dataframe()
        df = df.groupby(['name', 'hidden_size', 'learning_rate']).agg({'value':['count', 'min', 'mean', 'max']})
        df.columns = df.columns.map('_'.join)
        df = df.reset_index()
        print(df)

        # Plot grid search data
        Plotter(runs_logs).show_1d(query="hidden_size==4 and name=='time_left'")

Local CPU time mult = 0.80
Train data is  <class 'list'>  Lenth  8192
['hola', 'hola', 'hello', 'hola', 'hola', 'hola', 'hola', 'hello', 'hola', 'hola', 'hola', 'hola', 'hello', 'hola', 'hello', 'hello', 'hola', 'hola', 'hello', 'hello', 'hola', 'hello', 'hola', 'hello', 'hello', 'hello', 'hello', 'hola', 'hola', 'hello', 'hello', 'hello', 'hola', 'hello', 'hola', 'hola', 'hello', 'hola', 'hola', 'hello', 'hola', 'hello', 'hello', 'hello', 'hello', 'hello', 'hola', 'hello', 'hello', 'hello', 'hola', 'hello', 'hello', 'hola', 'hola', 'hello', 'hola', 'hola', 'hello', 'hello', 'hello', 'hola', 'hello', 'hello', 'hola', 'hello', 'hello', 'hello', 'hello', 'hello', 'hola', 'hello', 'hello', 'hola', 'hola', 'hello', 'hola', 'hello', 'hello', 'hello', 'hello', 'hola', 'hello', 'hello', 'hola', 'hello', 'hello', 'hello', 'hello', 'hola', 'hola', 'hola', 'hola', 'hola', 'hello', 'hola', 'hola', 'hello', 'hello', 'hola', 'hola', 'hello', 'hello', 'hola', 'hello', 'hola', 'hello', 'hello', 'hell

AttributeError: ignored